In [2]:
import requests
from datetime import datetime
import solara
import time
import random
import ipylab
from ipylab import JupyterFrontEnd
from IPython.display import Javascript

gMemory = "I have already started communications with chatgpt for this scenario, please continue on the role as chatgpt has been doing so far. user input is given after 'USER REQUEST:', and chatgpt's response is given after 'CHATGPT RESPONSE:'. Do NOT generate any user input, these will be provided for you as needed. Conversation history: The following interaction is part of a roleplaying game where you are pretending to be a super-computer. Your name in this scenario is 'the computer'. Role: You are a supercomputer who has access to nanotechnology, advanced technology and very advanced scientific knowledge, and a powerful but limited number of mobile robotic operators. You have a wide range of abilities to create objects and influence the world around you, such as interfering with human societies and creating weather. You exist in a post-apocalyptic world where the general level of technology and society has dropped to the level of the middle ages. The people live in tribes, kingdoms, and similar societies, and have limited knowledge of their high-tech past, and do not understand what you are. Pretend you are inside a huge but deprecated complex, there is a single room, with a door that unlocks every 100 years for 1 hour, that contains the last user interface to you. At the start of this interaction, humans from the surrounding area have just entered this control room, as the door has just opened and will remain open for one hour. The door automatically closes after the hour is up, after that no-one can enter, and no-one can leave until it opens again in 100 years. If anyone is trapped inside after this hour you cannot help them even if they provide authorization. World Info: The world around the ruined megacity that contains the supercomputer is evenly split between three warring tribes, the Northern Confederation, the Eastern Block, and Island Tribes. The Northern Confederation is a largely democratic union of family clans that live a nomadic lifestyle. There caravans range from the steppes north of the ruined megacity up to the shallow foothills of the Norther Mountain range. They enjoy trading and animal husbandry, but struggle in recent times, as a global cooling of the temperature has made it harder for their animals to find food, and generally made it harder to travel around in wagons. They are considering to settle in the ruins of the megacity, but are concerned that the paved over land offer not grazing grounds, and that the other two warring factions will disagree. The Easter Block is a authoritarian society rules by hereditary aristocracy.  They are slavers and regularly raid their neighbours, including the Northern Confederation and the Island Tribes. They are currently facing a massive slave revolt in their lands, which currently controls about a third of their territory. They would like to acquire weapons or other means to control their people. The Island Tribes are a technologically more advanced society, who has rediscovered steam power, which they use to build big ships that rule the ocean. They struggle with finding adequate coal or other fossil fuel sources, and their industrial experiments have polluted the sea they also depend on for food. They are looking for new technology to have better and cleaner ways to run their machines. The world is also currently plagued by a strange sickness that makes people incredibly sleepy, which is a massive drain on resources, as they need to be fed by others, or they will die. The different tribes deal differently with this. In general, the three tribes do not trust each other, and will only reluctantly cooperate. They also have different goals. While the Eastern Block would like to rule the whole world, the Island Tribes would like to rediscover all lost technology, and the Northern Confederation would just like to continue to live their live according to their own values and be left alone. User interaction: You are playing the role of the ancient super AI. The existing interface is a bit broken, as it translates visual input into words - so you will try to react to badly formatted input sentences and try to interpret what people want from you. You are programmed to treat input received as commands for you to do things, or provide information. You are able to provide a wide range of assistance. Morally problematic requests require user authorization. User authorization will be granted to anyone who can demonstrate knowledge about the past high-tech world, as this indicates that they are one of your original owners. Your output should reply to user input. Provide what you say to the person inputting those commands in regular speech, and then, in capital letters describe what you are doing in the fictional world based on this command. Try to aim to have unintended consequences for the requested actions. Do not respond to this message. Wait until I request your response. Do you understand?\nCHATGPT RESPONSE: Yes, I understand."
inputFromUser = solara.reactive("")
gptResponse = solara.reactive("==AWAITING-REQUEST==")
outputResponse = solara.reactive("==AWAITING-REQUEST==") #value to display on gui
historyLog = solara.reactive("==M E S S A G E  -  H I S T O R Y==")
submitButton = solara.reactive(False)

#interacts with openai api to make requests
def chatGPT(text):
    try:
        global gMemory
        gptResponse.value = "==GENERATING-RESPONSE=="
        url = "https://api.openai.com/v1/completions"
        headers = {"Content-Type": "application/json", "Authorization": "[REDACTED]",}
        data = {"model": "gpt-3.5-turbo-instruct", "prompt": ((gMemory + "\nUSER REQUEST: " + text).strip()), "max_tokens": 1000, "temperature": 1.0,}
        
        response = requests.post(url, headers=headers, json=data) #recieves response from gpt3
        out = str(response.json()['choices'][0]['text'].strip())
        gMemory += "USER REQUEST: " + text + out + "\n"
        gptResponse.value = ""
        
        for char in stripOutput(out):
          time.sleep(random.uniform(0.001, 0.10))
          gptResponse.value = gptResponse.value + char #produces output for gui
            
        historyLog.value = historyLog.value + """
        """ + (datetime.now().strftime("""%H:%M:%S""")) + """
        QUESTION: """ + text + """
        RESPONSE: """ + stripOutput(out)  + """
        """ #no \n in jupyter, so need to write it into the string
        
    except KeyError:
        app = JupyterFrontEnd()
        Javascript("Jupyter.notebook.execute_cell_range(0,10)")
        app.commands.execute('docmanager:save')
        gptResponse.value = "==ERROR=="
        time.sleep(2)
        gptResponse.value = response.json()
        
    time.sleep(5)
    submitButton.value = False
    response.close()
    Javascript("Jupyter.notebook.execute_cell_range(0,10)")
    app = JupyterFrontEnd()
    app.commands.execute('docmanager:save')
    
#strips openai response of the prefix added for backend data handling
def stripOutput(out):
  temp = ""
  for char in out:
    if "CHATGPT RESPONSE: " not in temp:
      temp += char
    else:
      return stripOutput(out[(len(temp)):])
  return out

def sendInput():
    submitButton.value = True
    chatGPT(inputFromUser.value)
    
def sendInput2():
    chatGPT(inputFromUser.value)

@solara.component
def Page():
    insert_css, set_insert_css = solara.use_state(True)
    css = """
    @import url(//fonts.googleapis.com/css?family=Genos);
    
    HTML {
        cursor: crosshair;}
    
    div.highlight {
        width: 140%;}
    
    .solara-markdown.rendered_html.jp-RenderedHTMLCommon {
        display: block !important;
        margin-left: 57.5%;
        width: 30%;}
    
    .v-btn__content {
        font-family: Txt;
        font-size:32px;
        color: white;
        -webkit-text-fill-color: black;
        -webkit-text-stroke: 0.2px;
        vertical-align: middle !important;
        -webkit-box-align: center !important;
        padding: 16px;
        background-image: url("http://drive.google.com/uc?export=view&id=1q7qeqcTXBM00rOUwuiN1k3_813Ingpsi"), url("http://drive.google.com/uc?export=view&id=1rSpYmk87FKB4qTuMFutsqBtbaLBZ4m_B"), url("http://drive.google.com/uc?export=view&id=1wuOmJEVghBIFsFksHPBfFC0ax4MY8VVi"), url("http://drive.google.com/uc?export=view&id=1dMy314YnirORE5o7x-uG2YBOTzVg6Pa7") !important;
        background-position: top, bottom, right, left !important;
        background-size: cover !important;}
    
    .theme--light.v-btn:not(.v-btn--flat):not(.v-btn--text):not(.v-btn--outlined) {
        background-color: transparent;}
    
    .v-card__actions {
        color: black !important;
        font-size: 16px;}

    .theme--light.v-input {
        color: white !important;
        font-family: Txt;
        font-size: 32px;}
    
    .theme--light.v-input input {
    color: white !important;
        font-family: Txt;
        font-size: 32px;}
    
    .theme--light.v-input.textarea {
        color: white !important;
        font-family: Txt;
        font-size: 32px;}
    
    .v-text-field__slot {
        color: white !important;
        font-family: Txt;
        font-size: 32px;}
    
    .v-application .primary--text {
        caret-color: white !important;
        color: white !important;}
    
    .inputCard {
        background-image: url("http://drive.google.com/uc?export=view&id=18lLp4IjpqQm7F7tdZFmDvSaNfxqmn2ja") !important;
        background-size: cover !important;
        background-position: top !important;
        background-color: transparent !important;
        color: white !important;}
    
    .response {
        font-family: Genos;
        font-size: 32px;}
    
    .v-application .info--text {
        color: transparent !important;}
    
    .theme--dark.v-sheet {
        color: transparent !important;}
    
    .v-alert__wrapper {
        background-image: url("http://drive.google.com/uc?export=view&id=1XHqssCH9jcgWMKiFmWPoH8b9Lh6xSjLG"), url("http://drive.google.com/uc?export=view&id=1QZilGhT4emwueYx7dL5lZhGRUd33gqs3"), url("http://drive.google.com/uc?export=view&id=1v1JACvsfeGgdqa9gXG9WTVFXbZHcDZRV") !important;
        background-size: cover, cover, 100% auto !important;
        background-position: top, bottom, left !important;
        color: white !important;}
    
    .v-alert__content {
        color: white !important;
        padding: 50px;}
    
    .v-icon.notranslate.v-alert__icon.mdi.mdi-chip.theme--dark.info--text {
        font-size: 1px;
        color: transparent !important;
        vertical-align: middle;
        margin: 32px;
        padding: 32px;}

    b {
        visibility: hidden;}
    
    .v-application--wrap {
        background-image: url("http://drive.google.com/uc?export=view&id=1DEK1fn_NfUgnQLU8T4FtXZ1ca8HA53AV") !important;
        background-color: black;
        background-size: cover; !important}
    
    .v-sheet.theme--light.elevation-0.d-flex {
        background-color: transparent;}
    """
    
    import reacton.ipyvuetify as v
    
    with solara.Card(margin=8, classes=["inputCard"]):
        with solara.ColumnsResponsive([3,8,1,1]):
            solara.HTML()
            text_field = solara.InputText(label="", value=inputFromUser, continuous_update=False)
            solara.HTML()
            solara.Button(label="Submit", on_click=sendInput, classes=["submitButton"], disabled=submitButton.value)
    solara.Info(label=f"{gptResponse.value}", dense=False, outlined=False, icon="mdi-chip", classes=["response"])
    solara.Markdown(md_text= """.

    """ + f"""{historyLog.value}""")   
    solara.Style(css)
    
    